In [1]:
import os
import re
import random   
import io
import time
from shutil import move

import pdfminer
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams

from pdf2image import convert_from_path
from PIL import Image, ImageDraw, ImageFont

from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup

import extractors
from utils import createDirIfNotExist, createDir, get_files
from words_handler import parse_obj, highlightObjects, DPI_SCALE, STANDART_DPI
from object_handlers import handleNumbers, handlePersons, handleDates, handleMoney, handleOrgs

# Markup docs

In [2]:
def saveToXML(objects, docName, images, xml, maxPageNum, processedText, text_lines, drawnBoxes, xmlsBoxes, border_colors):  
  font = ImageFont.truetype("Arsenal-Regular.otf", 20)

  unique_values = set()
  objects = [o for o in objects
      if (o['text_value'], o['bbox']) not in unique_values
      and not unique_values.add((o['text_value'], o['bbox']))]

  nums_objs = handleNumbers(objects, images[0].size[1])
  persons_objs = handlePersons(objects, images[0].size[1], processedText, text_lines, drawnBoxes, xmlsBoxes, 0, border_colors)
  dates_objs = handleDates(objects, images[0].size[1])
  money_objs = handleMoney(objects, images[0].size[1])
  orgs_objs = handleOrgs(objects, images[0].size[1], images[0].size[0])
  
  # objects = nums_objs + dates_objs + persons_objs + orgs_objs
  objects = persons_objs + dates_objs + nums_objs + money_objs + orgs_objs
  
    
  for pageNum in range(0, maxPageNum):
    im_height = images[pageNum].size[1]
    image_drawer = ImageDraw.Draw(images[pageNum])
    
    for obj in objects:
      if obj["pageNum"] != pageNum:
        continue
      
      bbox = (obj["bbox"][0], int(im_height - obj["bbox"][1] - pageNum * 10000),
          obj["bbox"][2], int(im_height - obj["bbox"][3] - pageNum * 10000))

      image_drawer.rectangle((bbox[0], bbox[1], bbox[2], bbox[3] + random.randint(2, 10)), 
                  outline=border_colors[obj['field_type']], width=3)
                  # outline=obj["border_color"], width=3)
      
      image_drawer.rectangle((bbox[0], bbox[3], bbox[2], bbox[3]-20), fill="white")

      image_drawer.text((bbox[0], bbox[3]-20), obj['field_type'] + " | " + obj["text_value"], 
                font = font, fill=border_colors[obj['field_type']])

      xml.write('  <{field_type} value="{value}" confidence="100" page="{page}" left="{x}" top="{y}" width="{w}" height="{h}"/>' \
            .format(field_type=obj['field_type'], value=obj["text_value"].replace('"', '').replace('<', '').replace('>', ''),
                x=bbox[2], y=bbox[1], w=bbox[2]-bbox[0], h=bbox[1]-bbox[3], page=pageNum)
            + '\n')
      
    images[pageNum].save("results/" + docName + "_" + str(pageNum) + ".jpg", "JPEG")
    

In [4]:
if __name__ == '__main__':
  MODEL_NAME = 'model'
  PDF_PATH = "input/"
  HANDLED_PATH = "handled/"
  SLEEP_TIME = 5

  border_colors = {
    'Number': 'red',
    'Date': 'deepskyblue',
    'Org': 'blue',
    'Money': 'green',
    'Debtor': 'orange',
    'Person': 'black',
    'Judge': 'yellow',
    'DebtorDate': 'purple',
    'LetterNumber': 'pink',
    'LetterDate': 'gray', 
    'PassportNum': 'red',
    'PassportSeries': 'red'
  }

  la_params = LAParams()
  la_params.line_margin = 1.6
  la_params.boxes_flow = 0.5

  createDirIfNotExist(PDF_PATH)
  createDirIfNotExist(HANDLED_PATH)
  createDir("results/", ".jpg")
  createDir("xmls/", ".xml")

  navec = Navec.load('vocab.tar')
  ner = NER.load(MODEL_NAME + '.tar')
  ner.navec(navec)

  while True:
      
    for doc_name in get_files(PDF_PATH, ".pdf"):
      images = convert_from_path(PDF_PATH + doc_name, dpi = STANDART_DPI * DPI_SCALE)

      fp = open(PDF_PATH + doc_name, 'rb')
      parser = PDFParser(fp)
      document = PDFDocument(parser)
      
      xml = io.open("xmls/" + doc_name.replace('.pdf', '') + ".xml", "w", encoding="utf-8")
      
      xml.write('<?xml version="1.0" encoding="UTF-8"?>' + '\n')
      xml.write('<idcard>' + '\n')

      drawnBoxes = []
      xmlsBoxes = []
      maxPageNum = 0
      
      for pageNum, page in enumerate(PDFPage.create_pages(document)):
        actName = doc_name.split(".")[0] + "_" + str(pageNum) 
        print(actName)

        text_lines = []
        
        rsr_mgr = PDFResourceManager()
        device = PDFPageAggregator(rsr_mgr, laparams=la_params)
        
        interpreter = PDFPageInterpreter(rsr_mgr, device)
        interpreter.process_page(page)
        
        layout = device.get_result()
        rawText = parse_obj(layout._objs, text_lines)

        processedText = re.sub(r"__+", r" ", rawText)

        # print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
        # if len(processedText) == 0:
        #   print("No text")
        # else:
        #   print(processedText)
        # print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<\n")

        if len(processedText) == 0:
          break

        markup = ner(processedText)
            
        extracted_nums = extractors.extractNums(processedText)
        highlightObjects(extracted_nums, 'Number', text_lines, drawnBoxes, xmlsBoxes, pageNum, border_colors)

        extracted_dates = extractors.extractDates(processedText)
        highlightObjects(extracted_dates, 'Date', text_lines, drawnBoxes, xmlsBoxes, pageNum, border_colors)

        extracted_orgs, extracted_persons = extractors.extractOrgs(markup)
        highlightObjects(extracted_orgs, 'Org', text_lines, drawnBoxes, xmlsBoxes, pageNum, border_colors)
        # highlightObjects(extracted_persons, 'Person')

        extracted_persons = extractors.extractPersons(processedText, markup)
        highlightObjects(extracted_persons, 'Person', text_lines, drawnBoxes, xmlsBoxes, pageNum, border_colors)

        extracted_debtors = extractors.extractDebtors(processedText)
        highlightObjects(extracted_debtors, 'Debtor', text_lines, drawnBoxes, xmlsBoxes, pageNum, border_colors)

        extracted_money = extractors.extractMoney(processedText)
        highlightObjects(extracted_money, 'Money', text_lines, drawnBoxes, xmlsBoxes, pageNum, border_colors)

        # extracted_addresses = extractors.extractAddresses(text_lines)
        # print(extracted_addresses)

        extracted_passport_series = extractors.extractPassportSeries(processedText)
        highlightObjects(extracted_passport_series, 'PassportSeries', text_lines, drawnBoxes, xmlsBoxes, pageNum, border_colors)

        extracted_passport_nums = extractors.extractPassportNums(processedText)
        highlightObjects(extracted_passport_nums, 'PassportNum', text_lines, drawnBoxes, xmlsBoxes, pageNum, border_colors)

        maxPageNum += 1

        
      saveToXML(xmlsBoxes, doc_name.replace('.pdf', ''), images, xml, maxPageNum, processedText, text_lines, drawnBoxes, xmlsBoxes, border_colors)
      xml.write('</idcard>' + '\n')
      xml.close()

      fp.close()
      # move(PDF_PATH + doc_name, HANDLED_PATH + doc_name)

    time.sleep(SLEEP_TIME + 10000000)

1_0
2_0
3_0
4_0
6_0
7_0
9_0
Scan2021-12-08_145113_0
Scan2021-12-08_145113_006_0
Scan2021-12-08_145113_007_0
Scan2021-12-08_145113_012_0
Scan2021-12-08_145249_0
Scan2021-12-08_145249_002_0
Scan2021-12-08_145249_003_0
Scan2021-12-08_145249_005_0
Scan2021-12-08_145249_007_0
Scan2021-12-08_145249_010_0
Scan2021-12-08_145249_012_0
Scan2021-12-08_145249_013_0
Scan2021-12-08_145249_015_0
Scan2021-12-08_145249_016_0
Scan2021-12-08_145249_018_0
Scan2021-12-08_145249_020_0
Scan2021-12-08_145249_021_0
Scan2021-12-08_145249_023_0
Scan2021-12-08_145249_025_0
Scan2021-12-08_145249_026_0
Scan2021-12-08_145249_027_0
Scan2021-12-08_145420_0
Scan2021-12-08_145420_1
Scan2021-12-08_145420_001_0
Scan2021-12-08_145420_001_1
Scan2021-12-08_145420_002_0
Scan2021-12-08_145420_002_1
Scan2021-12-08_145420_006_0
Scan2021-12-08_145420_006_1


KeyboardInterrupt: 